We compute in this worksheet the orbifold fundamental group $G_1$ of Proposition 4.3 in the paper `Torsion divisors of plane curves with maximal flexes and Zariski pairs` by E. Artal, S. Bannai, T. Shirane and H. Tokunaga. As for other groups we use the package `sirocco` by M.Á Marco and M. Rodríguez. Since we need to know which are the meridians of the irreducible components (including the line at infinity), instead of using the method `fundamental_group` we need to use internal functions of `sirocco` in order to obtain the braid monodromy for a projection. In order to do that, we need to use some functions constructed by C. Alquézar, which will be part of `sirocco` in forthcoming versions. They are introduced in the next cell, which is hidden (click at the three points to see it).

In [ ]:
from sage.schemes.curves import zariski_vankampen as zv

import numpy as np

from matplotlib import pyplot as plt

from scipy.spatial import Voronoi, voronoi_plot_2d

import copy

def get_voronoi(points):
    discpoints = np.array([(CC(a).real(), CC(a).imag()) for a in points])
    added_points = 3 * abs(discpoints).max() + 1.0
    configuration = np.vstack([discpoints, np.array([[added_points, 0], [-added_points, 0],
                                               [0, added_points], [0, -added_points]])])
    V = Voronoi(configuration)
    return (V)

class Graph:

    def __init__(self, vor, clockwise=False):
        r"""
            Constructor
        """
        self.vertices = np.array(copy.deepcopy(vor.vertices))
        self.edges = [copy.deepcopy(x) for x in vor.ridge_vertices if -1 not in x]
        self.regions = [copy.deepcopy(x) for x in vor.regions if -1 not in x]
        self.regions = [copy.deepcopy(x) for x in self.regions if x]  # remove empty list
        self.sort_all_regions(clockwise)
        self.boundary = self.get_boundary_vertices(clockwise)
        self.plot = False
        self.fix_axes = True

    def copy(self):
        return (copy.deepcopy(self))

#     def get_boundary_graph(self):

#         vertices = self.vertices
#         edges = self.get_boundary_edges()
#         regions = [self.get_boundary_vertices()]
#         data = []
#         return Graph(data)

    def get_boundary_edges(self):
        r"""
            Return
        """
        boundary = []
        for e in self.edges:
            count = 0
            for r in self.regions:
                if self.is_in_region(e, r):
                    count += 1
                if count > 1:
                    break
            if count < 2:
                boundary.append(e)
        return (boundary)

    def get_boundary_vertices(self, clockwise=False):
        r"""
            Return
        """
        vertices = []
        for e in self.get_boundary_edges():
            vertices += e
        boundary = list(set(vertices)) # remove redundancies
        boundary = self.sort_region_vertices(boundary, clockwise)
        return (boundary)

#     def get_envelope(self):
#         r"""
#             Return the smallest square that contains the graph
#         """
#         vmin = np.amin(self.vertices)
#         vmax = np.amax(self.vertices)
#         # corners
#         bottom_left = [vmin, vmin]
#         bottom_right = [vmin, vmax]
#         top_left = [vmin, vmax]
#         top_right = [vmax, vmax]
#         # list of corners
#         points = []
#         points.append(bottom_left)
#         points.append(bottom_right)
#         points.append(top_right)
#         points.append(top_left)
#         return points

    def is_in_region(self, edge, region):
        r"""
            Return true if the edge is in the boundary of the region
        """
        isIn = True
        for v in edge:
            if v not in region:
                isIn = False
                break
        return (isIn)

    def get_region_boundary(self, region):
        edges = []
        for i in range(len(region)-1):
            vertex1 = region[i]
            for j in range(i+1, len(region)):
                vertex2 = region[j]
                edge = [vertex1, vertex2] if vertex1 < vertex2 else [vertex2, vertex1]
                if edge in self.edges:
                    edges.append(edge)
        return (edges)

    def get_region_coords(self, region):

        coords = []
        for v in region:
            coords.append(self.vertices[v])
        return (coords)

    def is_ccw(self, region):
        coords = self.get_region_coords(region)
        coords1= copy.deepcopy(coords)
        miny=min([_[1] for _ in coords1])
        for a in coords1:
            a[1]=a[1]-miny+1
        sum = 0
        for i in range(len(coords1)):
            p0 = coords1[i]
            p1 = coords1[(i+1)%len(coords1)]
            sum += (p1[0]-p0[0])*(p1[1]+p0[0])
        return (sum < 0)

#     def is_ccw(self, region):
#         coords = self.get_region_coords(region)
#         ring = LinearRing(coords)
#         if ring.is_ccw:
#             return True
#         else:
#             return False

    def get_regions_by_edge(self, e):

        if e[1] < e[0]:
            e.reverse()
        regions = []
        for r in self.regions:
            boundary = self.get_region_boundary(r)
            if e in boundary:
                regions.append(r)
                if len(regions) == 2:
                    break
        return (regions)

    def get_consecutive_edge(self, edge, common_vertex, list_edges):
        for e in list_edges:
            if e != edge and common_vertex in e:
                return (e)

    def remove_edge(self,edge):
        self.edges.remove(edge)

    def remove_region(self, region, edge):
        new_graph = self.copy()
        if region in new_graph.regions:
            graph_boundary = new_graph.get_boundary_edges()
            region_boundary = new_graph.get_region_boundary(region)
            candidates = []
            for e in region_boundary:
                if e in graph_boundary:
                    candidates += [e]
            connectedToEdge = [edge]
            findmore = True
            while findmore:
                findmore = False
                for e in connectedToEdge:
                    if e in candidates:
                        candidates.remove(e)
                for e1 in connectedToEdge:
                    aux = [e for e in candidates if e1[0] in e or e1[1] in e]
                if len(aux) > 0:
                    findmore = True
                connectedToEdge += aux
            for e in connectedToEdge:
                if e in new_graph.edges:
                    new_graph.edges.remove(e)
            new_graph.regions.remove(region)

        return (new_graph)

    def sort_region_vertices(self, region, clockwise=False):

        # firts get the region boundary
        edges = self.get_region_boundary(region)
        # select the first edge of the list
        e = edges[0]
        # create the new list (initially empty)
        new_ordering = []
        # the first vertex will be the first end of the edge 'e'
        new_ordering.append(e[0])
        # now follow the boundary of the region
        next_vertex = e[1]
        next_edge = self.get_consecutive_edge(e, next_vertex, edges)
        #while next_vertex != new_ordering[0]:
        while next_vertex not in new_ordering:
            # add new vertex to list
            new_ordering.append(next_vertex)
            # get the other ending of the edge
            next_vertex = next_edge[0] if next_edge[0] != next_vertex else next_edge[1]
            next_edge = self.get_consecutive_edge(next_edge, next_vertex, edges)

        if clockwise:
            if self.is_ccw(new_ordering):
                new_ordering.reverse()
        else:
            if not self.is_ccw(new_ordering):
                new_ordering.reverse()

        return (new_ordering)

    def sort_all_regions(self, clockwise=False):
        for r in range(len(self.regions)):
            region = self.regions[r]
            self.regions[r] = self.sort_region_vertices(region, clockwise)

    def plot_graph(self, pause=0.1, color='k'):
        r"""
            Plot graph
        """
        plt.clf()
        plt.axis('off')
        for e in self.edges:
            v0 = self.vertices[e[0]]
            v1 = self.vertices[e[1]]
            x = [v0[0], v1[0]]
            y = [v0[1], v1[1]]
            vx = x[1]-x[0]
            vy = y[1]-y[0]
            angle = np.arctan2(vy,vx)
            plt.plot(x, y, color=color)
            v0 = self.vertices[e[0]]
            v1 = self.vertices[e[1]]
            plt.arrow(v0[0],v0[1],(v1[0]-v0[0])/2,(v1[1]-v0[1])/2,head_width=0.2, head_length=0.3, fc='k', ec='k')
        axes = plt.gca()
        if self.fix_axes:
            xmin = np.amin(self.vertices[:, 0])
            xmax = np.amax(self.vertices[:, 0])
            ymin = np.amin(self.vertices[:, 1])
            ymax = np.amax(self.vertices[:, 1])
            axes.set_xlim([xmin, xmax])
            axes.set_ylim([ymin, ymax])
            self.fix_axes = False
        for i in range(len(self.vertices)):
            plt.text(self.vertices[i][0],self.vertices[i][1],str(i), fontsize=12)
        for e in self.edges:
            v0 = self.vertices[e[0]]
            v1 = self.vertices[e[1]]
            midx = (v0[0]+v1[0])/2
            midy = (v0[1]+v1[1])/2
            plt.text(midx,midy,str(self.edges.index(e)+1), fontsize=15)
        
        plt.pause(pause)

def convertListEdges(ref, edges):
    indexList = []
    for e in edges:
        if e in ref:
            indexList.append(ref.index(e)+1)
        else:
            indexList.append(-(ref.index([e[1],e[0]])+1))
    return (indexList)

def find_basis(g, vertex_start, vertex_ending, generators=[], source_path=[], clockwise=False, initialGraph = None,count=0, plot=False):

    if count == 0:
        generators = []
        source_path = []
        #print(generators)
    edge = [vertex_start, vertex_ending] if vertex_start < vertex_ending else [vertex_ending, vertex_start]
    regions = g.get_regions_by_edge(edge)
    if regions:
        region = regions[0]  # only one region because the edge is on the boundary of the graph
        if plot:
            plt.cla()
            initialGraph.plot_graph(pause=0.1, color=(0, 0, 0, 0.2))
            g.plot_graph()
        
        ################################################################
        index = region.index(vertex_start)
        shift = index
        region = np.roll(region, -shift).tolist()
        ################################################################
        # closed path around the region of interest
        closedPathRegion = []
        for w in range(len(region)):
            w1 = region[w]
            w2 = region[(w+1)%len(region)]
            e = [w1, w2]
            closedPathRegion += [e]
        # geometric generator of the corresponding region from base point
        generator = dict({})
        generator['start'] = copy.deepcopy(source_path)
        generator['region'] = closedPathRegion
        reversedPath = copy.deepcopy(source_path)
        reversedPath.reverse()
        for e in reversedPath:
            e.reverse()
        generator['end'] = reversedPath
        if plot:
            for e in source_path:
                g.colour_edge(e, pause=0.1)

        if plot:
            g.colour_region(region)

        for e in closedPathRegion:
            if plot:
                g.colour_edge(e, pause=0.1)
            
        # add generator to the list of generators
        generators += [generator]

        source_path += [[vertex_start, vertex_ending]]
        boundary_edges = g.get_boundary_edges()
        next_edge = g.get_consecutive_edge(edge, vertex_ending, boundary_edges)
        vertex_start = vertex_ending
        vertex_ending = next_edge[0] if next_edge[0] != vertex_ending else next_edge[1]
        while (g.is_in_region(next_edge, region) or len(g.get_regions_by_edge(next_edge)) == 0) and len(g.regions) > 1:
            source_path += [[vertex_start, vertex_ending]]
            if len(g.get_regions_by_edge(next_edge)) == 0:  # edge does not belong to any region
                g.remove_edge(next_edge)
            next_edge = g.get_consecutive_edge(next_edge, vertex_ending, boundary_edges)
            vertex_start = vertex_ending
            vertex_ending = next_edge[0] if next_edge[0] != vertex_ending else next_edge[1]

        region = np.roll(region, shift).tolist()  # restore the default value of the region
        count += 1
        find_basis(g.remove_region(region, edge), vertex_start, vertex_ending, generators=generators, source_path=source_path, clockwise=clockwise, initialGraph=initialGraph, count=count, plot=plot)

    return (generators)

def get_basis(g, clockwise=False, plot=False):
    g.sort_all_regions(clockwise)
    boundary_vertices = g.get_boundary_vertices(clockwise)
    v0 = boundary_vertices[0]
    v1 = boundary_vertices[1]
    generators = find_basis(g, v0, v1, initialGraph=g, plot=plot)
    return (generators)


We introduce the curve. The equations are in a number field; $f$ is the cubic, $T_1$ is the equation of one triangle (which contains the line at infinity) and $T_2$ is the equation of another triangle (this polynomial has three linear factors in an extension of the number field).

In [ ]:
R0.<t0>=QQ[]
p0=t0^2 - 3*t0 + 9
a0=p0.roots(QQbar)[0][0]
L.<u0>=NumberField(p0,embedding=a0)
S1.<x,y>=L[]
f=x^2*y+y^2+x
T1=x*y
T2=x^3 + y^3 + u0*x*y + 1
F=(f*T1*T2)(x=x+y)

The variable `disc` contains the points in the discriminant and the variable `segs` the edges where the braids needed to obtain the fundamental group are computed. The variable `g` contains the information of the Voronoi diagram used to compute the braids

In [ ]:
disc = zv.discrim(F)
segs = zv.segments(disc)
g=Graph(get_voronoi(disc))

Since we are combining the methods of `sirocco` and the new functions by C. Alquézar, we check that they are compatible.

In [ ]:
gv=[list(_) for _ in list(g.vertices)]
ge=[tuple([gv[a][0]+I*gv[a][1] for a in b]) for b in g.edges]
ge==segs

The braids associated to the edges in `segs` (or `ge`) have been computed in a more powerful computed. The input is the list Tietze words of each braid, which are converted into actual braids.

In [ ]:
A=[(), (7, 6, 2, 5, 3, 4, 3, 5, 2, 6, 7), (7, 6, 7, 1, 5, 2, 4), (), (3, 6, 4, 2, 5, 1, 7, 6, 7), (6, 3, 2, -3, -6, -5, 6, 5, -2, -5), (4, -6, -7, -6, -5, -6, -5, 3, -5), (4, -3, -1, 5, -4, -2, -5), (-4, -3, -2), (6, 2, 7, 6, 5, 6), (-4, 5, -1, -3, -4), (1, 4, 5, 6), (-5, 3, -4), (4, 5, 3), (-3, 2), (7, -6, 4), (4, 6, -4, 7, 6, 4, 5, 4), (-6, -7, 6, 5, 6, -2), (5, -3, 4), (-4, 6, 7, 6, 5, 4), (-6, -4, -7), (3, -2), (4, 3, 2), (), (-1,), (-4, -5, -6, -4, -5, -3), (4, 3, 2, 1, 3, 2, -3, -4), (-5, -4, -1), (4, -6, -3, -5, 1, 2, 3, -1, -3), (5, -4, 1, 4), (3, -2, -1, -5, -3, -6), (-3, 6, -2, 5, -4, -5), (2, -5, 6, 5), (5, 4, -6, -5), (3,), (2, 5, -3), (), (-7, -6, -5, -4, -2, -5), (1, 7, 6), (-6, -1, -5, -7, 5, 4, 5, 6, 5, -4), (-4, 6, 3, 4, -7), (3, 1, 4, -3, -4, -2, -1, 4, 5, -4, -5, -7, -2), (-6, -3, -4, -7)]
B=BraidGroup(8)
tr1=[B(_) for _ in A]

We produce a geometric basis for the fundamental group of the complement of the discriminant. This is done as a list of dictionnaries for the labels `'start'`, `'region'` and `'end'`. Each one is a list of integer: each integer is associated to a generator of a free group with basis the edges of the Voronoi diagram. The element for `'start'` represents a path from the base point to a point in a region;
the element for `'region'` is a closed path that runs counterclockwise the corresponding region; the elemento for `'end'` is the inverse of the elemento for '`start'`.

The product of the geometric basis (from last to first) is the counterclocwise boundary. The list `tr1` allows to define a morphism from this free group to the braid group. The images of the elements in the basis are the elements of braid monodromy.

In [ ]:
base0=get_basis(g)
for gen in base0:
    gen['start'] = convertListEdges(g.edges,gen['start'])
    gen['region'] = convertListEdges(g.edges,gen['region'])
    gen['end'] = convertListEdges(g.edges,gen['end'])
FG=FreeGroup(len(g.edges))
base1=[]
for gen in base0:
    gen1={a:FG(gen[a]) for a in gen.keys()}
    base1.append(gen1)

In [ ]:
bordeV=g.get_boundary_vertices()
bordeE=[[bordeV[i],bordeV[(i+1)%(len(bordeV))]] for i in [0..len(bordeV)-1]]
bordeE1=convertListEdges(g.edges,bordeE)
infinito=FG(bordeE1)
infinito

In [ ]:
libre2braid=FG.hom(tr1,B)

We compute the loops in the complement of the discriminant and the braids.

In [ ]:
lazos=[a['start']*a['region']*a['end'] for a in base1]
trenzas=[libre2braid(a) for a in lazos]
len(trenzas)

In [ ]:
[_.exponent_sum() for _ in trenzas]

There is a small gap in the above code and sometimes the loops of regions are clockwise. We inverse those ones and restart. .We check the product condition with the boundary

In [ ]:
for i in [3,-2,-1]:
    base1[i]['region']=base1[i]['region']^-1
lazos=[a['start']*a['region']*a['end'] for a in base1]
trenzas=[libre2braid(a) for a in lazos]
w=FG.one()
for a in lazos:
    w=a*w
w==infinito

In [ ]:
[_.exponent_sum() for _ in trenzas]

In fact it is more convenient to keep also the decomposition.

In [ ]:
puiseux0=[[libre2braid(a['start']),libre2braid(a['region'])] for a in base1]

We want to modify the above decomposition in order to have positive braids for the middle braids. The result will be kept in the list `puiseux`. Each element of the list `puiseux` have four entries: two braids $\tau_1,\tau_2$ ($\tau_2$ is a positive algebraic braid) and two numbers $m,n$. The braid associated to the basis element is $\tau_1\cdot\tau_2\cdot\tau_1^{-1}$; $m$ is the first strand involved in $\tau_2$ and $n$ is the number of strands involved in $\tau_2$.

In order to keep this computations as short as possible, we need to manipulate the braids one by one.

In [ ]:
puiseux=[]

The first braid has the following decomposition, which is added to the new list `puiseux`.

In [ ]:
u=puiseux0[0][1]
v=B([-5, -4, -3])
w=B([2])
show(u==v*w/v)
v=puiseux0[0][0]*v
v.Tietze()

In [ ]:
puiseux.append([v,w,2,2])

We repeat the process.

In [ ]:
puiseux0[1][1].Tietze()

In [ ]:
u=puiseux0[1][1]
v=B([4,-6,-7])
w=B((6, 5, 6))^2*B((6,))^2
u==v*w/v

In [ ]:
puiseux0[1][0].Tietze()

In [ ]:
v=B([3, -5])
w=B((5, 4, 5))^2*B((4,))^2
trenzas[1]==v*w/v

Guardar : (6,5,4) actuando en (6,5,6)^2*(5)^2 da (5,4,5)^2*(4)^2

In [ ]:
puiseux.append([v,w,4,3])

In [ ]:
puiseux0[2][1].Tietze()

In [ ]:
u=puiseux0[2][1]
v=B([7])
w=B((6, ))^2
u==v*w/v

In [ ]:
puiseux0[2][0].Tietze()

In [ ]:
v=B([3, 4, 5, 6])
w=B((7, ))^2
trenzas[2]==v*w/v

In [ ]:
puiseux.append([v,w,7,2])

In [ ]:
print(puiseux0[3][1].Tietze())

In [ ]:
u=puiseux0[3][1]
v=B([])
w=B((5, ))^2
u==v*w/v

In [ ]:
puiseux0[3][0].Tietze()

In [ ]:
v=B([-1, -2, 3])
w=B((4, ))^2
trenzas[3]==v*w/v

In [ ]:
puiseux.append([v,w,4,2])

In [ ]:
len(puiseux)

In [ ]:
print(puiseux0[4][1].Tietze())

In [ ]:
u=puiseux0[4][1]
v=B([-4])
w=B(( 5,  ))
u==v*w/v

In [ ]:
print(puiseux0[4][0].Tietze())

In [ ]:
v=B([-1, -1, -2, 3, 4])
w=B((5, ))
trenzas[4]==v*w/v

In [ ]:
puiseux.append([v,w,5,2])
len(puiseux)

In [ ]:
print(puiseux0[5][1].Tietze())

In [ ]:
u=puiseux0[5][1]
v=B([-7, -6])
w=B((4, 5, 4))^2* B((4,))^2
u==v*w/v

In [ ]:
print(puiseux0[5][0].Tietze())

In [ ]:
v=B([])
w=B((5, 4, 5))^2* B((5,))^2
trenzas[5]==v*w/v

In [ ]:
puiseux.append([v,w,4,3])
len(puiseux)

In [ ]:
print(puiseux0[6][1].Tietze())

In [ ]:
u=puiseux0[6][1]
v=B([-6])
w=B((7,))^2
u==v*w/v

In [ ]:
print(puiseux0[6][0].Tietze())

In [ ]:
v=B([3, -1, -1, 2])
w=B((1,))^2
trenzas[6]==v*w/v

In [ ]:
puiseux.append([v,w,1,2])
len(puiseux)

In [ ]:
print(puiseux0[7][1].Tietze())

In [ ]:
u=puiseux0[7][1]
v=B([4])
w=B((5,))^2
u==v*w/v

In [ ]:
print(puiseux0[7][0].Tietze())

In [ ]:
v=B([5, 6])
w=B((7,))^2
trenzas[7]==v*w/v

In [ ]:
puiseux.append([v,w,7,2])
len(puiseux)

In [ ]:
print(puiseux0[8][1].Tietze())

In [ ]:
u=puiseux0[8][1]
v=B([6,5])
w=B((4,))
u==v*w/v

In [ ]:
print(puiseux0[8][0].Tietze())

In [ ]:
v=B([-1, -1, -1, -1, -2, 3, -4])
w=B((5,))
trenzas[8]==v*w/v

In [ ]:
puiseux.append([v,w,5,2])
len(puiseux)

In [ ]:
print(puiseux0[9][1].Tietze())

In [ ]:
u=puiseux0[9][1]
v=B([])
w=B((1, ))^2
u==v*w/v

In [ ]:
print(puiseux0[9][0].Tietze())

In [ ]:
v=B([-1, -1, -1])
w=B((2, ))^2
trenzas[9]==v*w/v

In [ ]:
puiseux.append([v,w,2,2])
len(puiseux)

The following braid correspond to a vertical lines having two non-transversal points. It is the product of two commuting algebraic braids and we separate them.

In [ ]:
print(puiseux0[10][1].Tietze())

In [ ]:
u=puiseux0[10][1]
v1=B([4,7])
w1=B((6, 5, 6))^2*B((6, ))^2
v2=B([4, 3])
w2=B((2, 1, 2))^2*B((1, ))^2
u==v1*w1/v1*v2*w2/v2

In [ ]:
print(puiseux0[10][0].Tietze())

In [ ]:
v0=B((3, 6, 4,  5, 2, 1, 7, 6, 7, 4, 3, 2, 5, 4, 3, 5, 4, 6, 5, 4, 1, 7, 6, 2, 5, 3, 4, 3, 5, 2, 6, 7, 4, 5, 6, 4, 5, 3, 4, 5, 7, 6, 7, 5, 4, 3, 2, -4, -4, 3, 1, 6, 5, 4, 3, 2))
v0==puiseux0[10][0]

In [ ]:
v1=B([-1, -1, -7, -6, 5, -4, -1, 2, 2, 3, 2])
w1=B((2, 1, 2))^2*B((1, ))^2
v2=B([-1, -1, -1, -1, -2, -2, 5])
w2=B((4, 3, 4))^2*B((4, ))^2
trenzas[10]==v1*w1/v1*v2*w2/v2

In [ ]:
puiseux.append([v1,w1,1,3])
puiseux.append([v2,w2,3,3])
len(puiseux)

In [ ]:
print(puiseux0[11][1].Tietze())

In [ ]:
u=puiseux0[11][1]
v=B([6, 5, 1, -2, 3])
w=B((4, ))
u==v*w/v

In [ ]:
print(puiseux0[11][0].Tietze())

In [ ]:
v0=B([3, 6, 4, 2, 5, 1, 7, 6, 7, 4, 3, 2, 5, 4, 3, 5, 4, 6, 5, 4, 1, 7, 6, 2, 5, 3, 4, 3, 5, 6, 7, 4, 5, 6, 4, 2, 3, 4, 5, 4, 5, 6, 7, 5, 4, 3, 2, 6, 5, 4, 3, 2, 2, 1, 2, -5, -5, 6, 4, 4, 5])
v0==puiseux0[11][0]

In [ ]:
v=B([-1, -1, -1, 5, -7, 6, 4, -5, 2, 3, -1, 2, 1, 1, 4, 3, 3])
w=B((2, ))
trenzas[11]==v*w/v

In [ ]:
puiseux.append([v,w,2,2])
len(puiseux)

In [ ]:
print(puiseux0[12][1].Tietze())

In [ ]:
u=puiseux0[12][1]
v=B([])
w=B((2,   ))^2
u==v*w/v

In [ ]:
print(puiseux0[12][0].Tietze())

In [ ]:
v=B([5, 4, 6, 3, 2, 1, 5, 4, 3, 2, 5, 4, 3, -4, -4, -4, 5, -4, 3, 7, 6, 5])
w=B((4,   ))^2
trenzas[12]==v*w/v

In [ ]:
puiseux.append([v,w,4,2])

In [ ]:
len(puiseux)

In [ ]:
print(puiseux0[13][1].Tietze())

In [ ]:
u=puiseux0[13][1]
v=B([1, 2, 3, 4])
w=B((5,))
u==v*w/v

In [ ]:
print(puiseux0[13][0].Tietze())

In [ ]:
v=B([5, 4, 3, 2, 1, 1, 2, 3, 4, 5, -7, 6])
w=B((7,))
trenzas[13]==v*w/v

In [ ]:
puiseux.append([v,w,7,2])

We have all the Puiseux decompositions of the braids. We need to identify to which component each meridian belongs. We see that the generators $\mu_2,\mu_6,\mu_7$ correspond to the cubic. The other ones (and the meridian at infinity) correspond to the lines. We may distinguish the two triangles but we will do it later. This is done looking at the orbits by the monodromy action.

In [ ]:
permutaciones=PermutationGroup([_.permutation() for _ in trenzas])
permutaciones.orbits()

We start with the free group corresponding to the fundamental group of the complement of the curve in a vertical line. We add the orbifold relations: the cubes of the meridians of the lines and the 9th power of the meridian of the cubic.

In [ ]:
FL8=FreeGroup(8)
L8=[FL8(_)^3 for _ in [[1], [3], [4], [5], [8],[1..8]]]+[FL8([2])^9]

We add the relations of the braid monodromy using the Puiseux decomposition.

In [ ]:
for tau1,tau2,m0,m1 in puiseux:
    for j in [m0..m0+m1-1]:
        v=((FL8([j])*tau2)/FL8([j]))*tau1^-1
        L8.append(v)

In [ ]:
G=FL8/L8

We simplify the presentation and we keep track of the meridians. We see that this orbifold group is finite. As it is bigger than the abelianization, the group is not abelian.

In [ ]:
hom1=G.simplification_isomorphism()
G1=hom1.codomain()

In [ ]:
G1.order()

In [ ]:
ab=G1.abelian_invariants()
print(ab)
prod(ab)

We express the meridians in terms of the generators of $G_1$.

In [ ]:
meridianos=[hom1(G(_)).Tietze() for _ in [[1],[3],[4],[5],[8],[-8..-1],[6]]]
meridianos

Since there are `GAP` functions which are not natively translated in `Sagemath` we translate some objects into `GAP`.

In [ ]:
G1gap=G1.gap()

In [ ]:
meridianosgap=[G1(_).gap() for _ in meridianos]
len(meridianosgap)

In [ ]:
hom=G1gap.IsomorphismPermGroup()
G1a=hom.Range()
G1gap.GeneratorsOfGroup()

In [ ]:
[hom.Image(_).Order() for _ in meridianosgap]

We check the commutators of the meridians. One of them, $v$, is the generator of the derived subgroup.

In [ ]:
G1gap.DerivedSubgroup().Size()

In [ ]:
conms={(i,j):hom.Image(meridianosgap[i]*meridianosgap[j]/meridianosgap[i]/meridianosgap[j]) for i in [0..5] for j in [i+1..6]}
for i in [0..4]:
    for j in [i+1..5]:
        print (i,j,conms[i,j].Order())

In [ ]:
v=conms[0,4]^-1

The commutations of the meridians allow us to identify the triangles. The meridians of distinct triangles pairwise commute. And the meridians in the same triangle do not commute. To match with the notations of Proposition 4.3, the meridians in $X$ are $0,5,4\mapsto x_1,x_2,x_3$ while the meridians in $Y$ are $1,2,3\mapsto y_1,y_2,y_3$. 

In [ ]:
X=[0,4,5]
Y=[1,2,3]

In [ ]:
for i in X:
    for j in X:
        if i<j:
            c=conms[i,j]
            print(c.Order(),c==v)

In [ ]:
for i in Y:
    for j in Y:
        if i<j:
            c=conms[i,j]
            print(c.Order(),c==v)

In [ ]:
for i in X:
    for j in Y:
        if i<j:
            print(conms[i,j].Order())
        if j<i:
            print(conms[j,i].Order())

We check which commutators equal $v$ or $v^{-1}$.

In [ ]:
for i in [0..4]:
    for j in [i+1..5]:
        if conms[i,j].Order()==3:
            print(i,j,v==conms[i,j])

In [ ]:
for i in [0..4]:
    for j in [i+1..5]:
        if conms[i,j].Order()==3:
            print(i,j,v==conms[i,j])

In [ ]:
for i in [0..5]:
    u=conms[i,6]
    if u.Order()==1:
        print(i,"trivial")
    elif u==v:
        print(i,"=v")
    else:
        print(i,"=inverse of v")

The cell below show the centrality of the derived subgroup.

In [ ]:
for a in G1gap.GeneratorsOfGroup():
    b=hom.Image(a)
    print ((b*v/b/v).Order())